# Data import and preprocessing

Data is imported from the Stanford University HIV Drug resistance database https://hivdb.stanford.edu/pages/genopheno.dataset.html

In [57]:
from urllib.request import urlretrieve
import os

# Setup Imports
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer, load_diabetes, load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display, Markdown, Latex

# Baseline Imports
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

import torch

from tabpfn import TabPFNClassifier, TabPFNRegressor
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier, AutoTabPFNRegressor


In [3]:
# Stanford databases:
# High quality and full datasets

urls = ["https://hivdb.stanford.edu/_wrapper/download/GenoPhenoDatasets/PI_DataSet.txt",
        "https://hivdb.stanford.edu/_wrapper/download/GenoPhenoDatasets/NRTI_DataSet.txt",
        "https://hivdb.stanford.edu/_wrapper/download/GenoPhenoDatasets/NNRTI_DataSet.txt",
        "https://hivdb.stanford.edu/_wrapper/download/GenoPhenoDatasets/INI_DataSet.txt",
        "https://hivdb.stanford.edu/_wrapper/download/GenoPhenoDatasets/PI_DataSet.Full.txt",
        "https://hivdb.stanford.edu/_wrapper/download/GenoPhenoDatasets/NRTI_DataSet.Full.txt",
        "https://hivdb.stanford.edu/_wrapper/download/GenoPhenoDatasets/NNRTI_DataSet.Full.txt",
        "https://hivdb.stanford.edu/_wrapper/download/GenoPhenoDatasets/INI_DataSet.Full.txt"]

data_storage = r"C:\Users\flori\Desktop\Masterarbeit\Models\data"

#importing hq urls
for url in urls:
    filename = url.split("/")[-1]

    urlretrieve(url, data_storage + "\\" + filename)



In [49]:
# table for the encoding of the resistance testing into three classes: "susceptible", "intermediate-level resistant", "high-level resistant" with lower and upper thresholds

thresholds = [
    [3, 15],    # FPV
    [3, 15],    # ATV
    [3, 15],    # IDV
    [9, 55],    # LPV
    [3, 6],     # NFV
    [3, 15],    # SQV
    [2, 8],     # TPV
    [10, 90],   # DRV
    [5, 25],    # X3TC
    [2, 6],     # ABC
    [3, 15],    # AZT
    [1.5, 3],   # D4T
    [1.5, 3],   # DDI
    [1.5, 3],   # TDF
    [3, 10],    # EFV
    [3, 10],    # NVP
    [3, 10],    # ETR
    [3, 10],    # RPV
]

# Define row and column names
index = ["FPV","ATV","IDV","LPV","NFV","SQV","TPV","DRV",
         "3TC","ABC","AZT","D4T","DDI","TDF",
         "EFV","NVP","ETR","RPV"]
columns = ["lower", "upper"]

# Create DataFrame
cutoff_df = pd.DataFrame(thresholds, index=index, columns=columns)

print(cutoff_df.head())

     lower  upper
FPV    3.0     15
ATV    3.0     15
IDV    3.0     15
LPV    9.0     55
NFV    3.0      6


In [58]:
# Reading in and processing high quality File

df = pd.read_csv("data\\PI_DataSet.txt", sep='\t')
#print(df)
df = df.iloc[:,1:-1]
#print(df2)

#Checking how much data is available for each drug
print(df.loc[:,"FPV":"DRV"].count())

#list of current drugs of the dataset
drugs = [drug for drug in list(df.columns) if not drug.startswith("P") ]

#creating the one hot encoding for the features
enc = OneHotEncoder(handle_unknown='error')

enc.fit(df.loc[:,[drug for drug in list(df.columns) if drug.startswith("P")]])

print(enc.categories_)


FPV    2246
ATV    1656
IDV    2297
LPV    1999
NFV    2344
SQV    2298
TPV    1345
DRV    1116
dtype: int64
[array(['-', '.'], dtype=object), array(['-', '.', 'Q*', 'V'], dtype=object), array(['-', '.', 'F', 'IT', 'IV', 'L', 'M', 'V'], dtype=object), array(['-', '.', 'A', 'D', 'P', 'PS', 'S', 'TA', 'TN', 'TP', 'TS'],
      dtype=object), array(['-', '.', 'F', 'LF', 'LH', 'LI', 'LV'], dtype=object), array(['-', '.', 'GV', 'P', 'R', 'S', 'W*', 'WC', 'WL', 'WLS', 'WR', 'WS',
       'X'], dtype=object), array(['-', '.', 'E', 'K', 'QH', 'QK', 'QR', 'R'], dtype=object), array(['-', '.', 'Q', 'R*', 'RG', 'RQ'], dtype=object), array(['-', '.', 'PAS', 'PL', 'PS', 'X'], dtype=object), array(['-', '.', 'C', 'CF', 'F', 'FI', 'FIV', 'FV', 'H', 'I', 'IF', 'IM',
       'IST', 'IT', 'IV', 'LF', 'LFI', 'LFV', 'LI', 'LIV', 'LR', 'LV',
       'M', 'R', 'S', 'V', 'VF', 'VI', 'X', 'Y'], dtype=object), array(['-', '.', 'I', 'IL', 'IT', 'L', 'LI', 'VA', 'VF', 'VG', 'VI',
       'VIL', 'VL'], dtype=object), 

In [61]:
#print(drugs)

#going through the drugs and splitting them to test and training depending on the drug

for drug in drugs:
    #print(drug)
    tmp_drugs = drugs.copy()
    #print(tmp_drugs)
    tmp_drugs.remove(drug)
    #print(tmp_drugs)
    last_col = list(df.columns)[-1]
    dataframe = df.drop(tmp_drugs, axis=1)

    #print(dataframe.head())

    dataframe = dataframe.dropna()

    # encoding the levels of susceptibility as 0 for susceptible, 1 as partly resistant and 2 as compeltly resistant
    dataframe.loc[dataframe[drug] < cutoff_df.loc[drug, "lower"], drug + "_level"] = 0
    dataframe.loc[dataframe[drug] >= cutoff_df.loc[drug, "upper"], drug + "_level"] = 2
    dataframe.loc[(dataframe[drug] >= cutoff_df.loc[drug, "lower"]) & (dataframe[drug] < cutoff_df.loc[drug, "upper"]), drug + "_level"] = 1

    #print(dataframe.head())

    X, y = dataframe.drop([drug, drug + "_level"], axis=1), np.array(dataframe[drug + "_level"])

    print(X)

    X_trafo = enc.transform(X).toarray()

    print(X_trafo)

    #print(y)
'''
    # Convert all categorical columns to numeric
    for col in X.select_dtypes(['category']).columns:
        X[col] = X[col].cat.codes

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
'''

     P1 P2 P3 P4 P5 P6 P7 P8 P9 P10  ... P90 P91 P92 P93 P94 P95 P96 P97 P98  \
0     -  -  -  -  -  -  -  -  -   -  ...   -   -   -   -   -   -   -   -   -   
1     -  -  -  -  -  -  -  -  -   -  ...   -   -   -   -   -   -   -   -   -   
2     -  -  -  -  -  -  -  -  -   -  ...   -   -   -   -   -   -   -   -   -   
3     -  -  -  -  -  -  -  -  -   R  ...   -   -   -   -   -   -   -   -   -   
4     -  -  -  -  -  -  -  -  -   I  ...   M   -   -   -   -   -   -   -   -   
...  .. .. .. .. .. .. .. .. ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..   
2389  -  -  -  -  -  -  -  -  -   I  ...   M   -   -   L   -   -   -   -   -   
2390  -  -  -  -  -  -  -  -  -   I  ...   -   -   -   -   -   -   -   -   -   
2391  -  -  -  -  -  -  -  -  -   I  ...   -   -   -   -   -   -   -   -   -   
2393  -  -  -  -  -  -  -  -  -   -  ...   M   -   -   L   -   -   -   -   -   
2394  -  -  -  -  -  -  -  -  -   I  ...   M   -   -   L   -   -   -   -   -   

     P99  
0      -  
1      -  
2     

"\n    # Convert all categorical columns to numeric\n    for col in X.select_dtypes(['category']).columns:\n        X[col] = X[col].cat.codes\n\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)\n"